In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_trunc
from datetime import datetime
import os
from dotenv import load_dotenv

class DeltaToClickHouse:
    def __init__(self, minio_endpoint, access_key, secret_key, clickhouse_url, clickhouse_user, clickhouse_password):
        self.spark = SparkSession.builder \
            .appName("Delta to ClickHouse") \
            .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
            .config("spark.hadoop.fs.s3a.access.key", access_key) \
            .config("spark.hadoop.fs.s3a.secret.key", secret_key) \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .config("spark.jars.packages", "com.clickhouse:clickhouse-jdbc:0.4.6") \
            .getOrCreate()

        self.jdbc_options = {
            "url": clickhouse_url,
            "user": clickhouse_user,
            "password": clickhouse_password,
            "driver": "com.clickhouse.jdbc.ClickHouseDriver"
        }

    def get_max_ts_and_id_from_clickhouse(self, table, id_col):
        # Busca max timestamp e max id para filtro incremental mais preciso
        query = f"""
            (SELECT
                max(process_timestamp) as max_ts,
                max({id_col}) as max_id
            FROM {table}) AS max_table
        """
        df = self.spark.read.format("jdbc") \
            .options(**self.jdbc_options) \
            .option("dbtable", query) \
            .load()

        row = df.collect()[0]
        max_ts = row['max_ts'] if row['max_ts'] is not None else datetime(1900, 1, 1)
        max_id = row['max_id'] if row['max_id'] is not None else ''
        return max_ts, max_id

    def load_delta_and_filter(self, delta_path, max_ts, max_id, id_col):
        df = self.spark.read.format("delta").load(delta_path)
        # Trunca timestamp para segundos para comparar no mesmo nível de granularidade
        df = df.withColumn("process_timestamp_trunc", date_trunc("second", col("process_timestamp")))

        # Ajusta max_ts para truncar também, garantindo o filtro correto
        from pyspark.sql.functions import lit
        from pyspark.sql.types import TimestampType
        max_ts_trunc = max_ts.replace(microsecond=0)

        # Filtra linhas onde:
        # process_timestamp_trunc > max_ts_trunc
        # ou process_timestamp_trunc == max_ts_trunc e id_col > max_id
        df_filtered = df.filter(
            (col("process_timestamp_trunc") > lit(max_ts_trunc).cast(TimestampType())) |
            ((col("process_timestamp_trunc") == lit(max_ts_trunc).cast(TimestampType())) & (col(id_col) > max_id))
        ).drop("process_timestamp_trunc")

        # Opcional: deduplicar baseado no id_col para evitar duplicatas
        df_filtered = df_filtered.dropDuplicates([id_col])

        return df_filtered

    def write_to_clickhouse(self, df, table):
        df.write \
            .format("jdbc") \
            .mode("append") \
            .option("dbtable", table) \
            .options(**self.jdbc_options) \
            .save()

    def sync_table(self, delta_path, table, id_col):
        max_ts, max_id = self.get_max_ts_and_id_from_clickhouse(table, id_col)
        print(f"[{table}] Último timestamp no ClickHouse: {max_ts}, último ID: {max_id}")

        df_filtered = self.load_delta_and_filter(delta_path, max_ts, max_id, id_col)

        print(f"[{table}] Linhas a enviar: {df_filtered.count()}")
        self.write_to_clickhouse(df_filtered, table)
        print(f"[{table}] Dados inseridos com sucesso.")

    def run(self):
        self.sync_table("s3a://staging/fato_pedidos", "fato_pedidos", "ID_Pedido")
        self.sync_table("s3a://staging/dim_cliente", "dim_cliente", "ID_Cliente")
        self.sync_table("s3a://staging/dim_produto", "dim_produto", "Product_ID")

if __name__ == "__main__":
    load_dotenv()  # Carrega variáveis de ambiente do .env, se houver

    sync = DeltaToClickHouse(
        minio_endpoint="http://minio:9000",
        access_key="jGL83XVOmaZOGSKipZzb",
        secret_key="bUE3WUc0tvplgyss01XlMmpHSLsoZCkEomgCh93s",
        clickhouse_url="jdbc:clickhouse://clickhouse:8123/pedidos",
        clickhouse_user=os.getenv("CLICKHOUSE_USER"),
        clickhouse_password=os.getenv("CLICKHOUSE_PASSWORD")
    )

    sync.run()


[fato_pedidos] Último timestamp no ClickHouse: 1970-01-01 00:00:00, último ID: 
[fato_pedidos] Linhas a enviar: 25035
[fato_pedidos] Dados inseridos com sucesso.
[dim_cliente] Último timestamp no ClickHouse: 1970-01-01 00:00:00, último ID: 
[dim_cliente] Linhas a enviar: 1590
[dim_cliente] Dados inseridos com sucesso.
[dim_produto] Último timestamp no ClickHouse: 1970-01-01 00:00:00, último ID: 
[dim_produto] Linhas a enviar: 10292
[dim_produto] Dados inseridos com sucesso.
